In [134]:
import numpy as np
import pandas as pd
import twint
import time
from datetime import datetime
from datetime import timedelta, date
import json
from collections import Counter
from ast import literal_eval

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import nest_asyncio
nest_asyncio.apply()

### Change here

In [135]:
# ALL_USER_IDS_PATH = WHERE YOU STORE THE FILE WITH ALL CONSPIRACY USER IDS


#DELETE THIS:
ALL_USER_IDS_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Getting Conspiracy Hashtag Users/All User IDs.csv"

In [102]:
all_user_IDs = pd.read_csv(fr"{ALL_USER_IDS_PATH}")

all_user_IDs = all_user_IDs.set_index('ID')

## Clean tweets function

In [103]:
def clean_reply_to(obs):
    
    if len(obs) != 0:

        return [int(x['id']) for x in (obs)]
            
    
# vec_clean_reply_to = np.vectorize(clean_reply_to)

In [104]:
language_replacement_dict = {'en': 1, 'und':0}

def replace_0_or_1(x):
    if x not in (0,1):
        return np.nan
    else:
        return x

def clean_language(series):
    local_series = series.map(language_replacement_dict)
    
    return [replace_0_or_1(x) for x in local_series]

In [105]:
def clean_tweets(df):
    local_df = df
    
    
    # drop redundant cols
    local_df.drop(['created_at', 'timezone', 'cashtags', 'user_id_str', 'photos', 'video', 'thumbnail', 'translate', 'trans_src', 'trans_dest', 'name', 'search', 'day', 'hour', 'link', 'quote_url'], axis=1, inplace=True)
    
    
    # convert date to datetime
    local_df['date'] = pd.to_datetime(local_df['date'])
    
    
    # clean "reply_to" field
    local_df['reply_to'] = [clean_reply_to(x) for x in local_df['reply_to']]
    
    
    # clean "language field"
    local_df['language'] = clean_language(local_df['language'])
    
    
    return local_df


# Final Loop

### Change here

In [106]:
#PATH_ROOT = FOLDER WHERE YOU WANT TO SAVE TWEETS

#DELETE THIS:
#PATH_ROOT = "C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/All Conspiracy Tweeters' Tweets/"

In [107]:
def search_for_user(ID):
    '''
    ID = str(twitter user_i ID)
    '''
    c = twint.Config()
        
    c.User_id = ID
        
    c.Hide_output = True    
    
    # find shadow-banned accounts too - this apparently slows things down considerably
    c.Profile_full = True    
    
    c.Since = '2019-01-01'

    c.Pandas = True

    twint.run.Search(c)
    
    clean_tweets(twint.storage.panda.Tweets_df).to_csv(fr"{PATH_ROOT}{ID}_TWEETS.csv")

### Change here

In [108]:
for i, row in all_user_IDs.iterrows():
    
    if row['Searched'] != 1:
    
        search_for_user( str(row.name) )

        all_user_IDs['Searched'].loc[row.name] = 1

        if i % 100:    #save .csv every 100 users (essentially update progress)

            all_user_IDs.to_csv(ALL_USER_IDS_PATH, index=False)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


CRITICAL:root:twint.run:Twint:Feed:noData'globalObjects'
sleeping for 1.0 secs


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


# Split All User IDs for Concurrent Searching

In [140]:
for mod_10 in list(range(0,10)): #every 10 observations
    
    all_user_IDs.iloc[mod_10::10].to_csv(fr"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Getting Conspiracy Hashtag Users/All User IDs - Split {mod_10}")